In [3]:
import requests
import pandas as pd
import json
import time

In [7]:
my_payload = {'number': '請輸入你的電話', 
              'email': '請輸入你的 mail',
              'password':'請輸入你的密碼' }

def 取得電子發票記錄(start=2011, end=2018, payload=my_payload, period=6):
    # 建立 session 保存以及登入資訊
    session = requests.Session()
    session.post('https://www.ecloudlife.com/login', data=payload)
    
    # 產生迭代日期
    dates = []
    for invoice_year in range(end-start+1):
        for invoice_period in range(period):
            dates.append(str(invoice_year+start)+str(invoice_period))
    
    # 迭代日期
    df_invoice_list=[]
    for date in dates:
        url = 'https://www.ecloudlife.com/invoice/my/get?period=' + date
        res = session.get(url)
        data_json = json.loads(res.text)
        
        for index in range(len(data_json['invoices'])):
            invoice_number = data_json['invoices'][index]['invoice_number']
            '20183&n=FQ22015064'
            invoice_detail = session.get('https://www.ecloudlife.com/invoicebody/get?p='+date+'&n='+invoice_number)
            invoice_detail_json = json.loads(invoice_detail.text)
            data_json['invoices'][index]['detail'] = invoice_detail_json
        
        df_invoice = pd.DataFrame(data_json['invoices'])        
        df_invoice_list.append(df_invoice)
        
        with open('data_invoice_'+date+'.json', 'w') as f:
            json.dump(data_json, f, ensure_ascii=False)
        
        print('output json file', 'data_invoice_'+date+'.json', 'OK')
        
        time.sleep(5)
        
        print(date, 'download OK and sleep 5 sec')
        
    data_combine = pd.concat(df_invoice_list)

    data_combine = data_combine.reset_index(drop=True)
    
    df_invoice_detail_list = []

    for row in range(len(data_combine)):
        df_invoice_detail = pd.DataFrame(data_combine['detail'][row])
        df_invoice_detail_list.append(df_invoice_detail)

    df_all_detail_table = pd.concat(df_invoice_detail_list).reset_index(drop=True)
    
    df_all_detail_table['amount'] = pd.to_numeric(df_all_detail_table['amount'], errors='coerce')
    df_all_detail_table['quantity'] = pd.to_numeric(df_all_detail_table['quantity'], errors='coerce')
    df_all_detail_table['unit_price'] = pd.to_numeric(df_all_detail_table['unit_price'], errors='coerce')

    return data_combine, df_all_detail_table

In [ ]:
df, df_detail = 取得電子發票記錄(start=2011, end=2018, payload=my_payload, period=6)

In [11]:
df.to_pickle('invoice.pickle') # 以後可以直接調用, 不需要再從網路下載一次
df_detail.to_pickle('invoice_detail.pickle')

output json file data_invoice_20110.json OK
20110 download OK and sleep 5 sec
output json file data_invoice_20111.json OK
20111 download OK and sleep 5 sec
output json file data_invoice_20112.json OK
20112 download OK and sleep 5 sec
output json file data_invoice_20113.json OK
20113 download OK and sleep 5 sec
output json file data_invoice_20114.json OK
20114 download OK and sleep 5 sec
output json file data_invoice_20115.json OK
20115 download OK and sleep 5 sec
output json file data_invoice_20120.json OK
20120 download OK and sleep 5 sec
output json file data_invoice_20121.json OK
20121 download OK and sleep 5 sec
output json file data_invoice_20122.json OK
20122 download OK and sleep 5 sec
output json file data_invoice_20123.json OK
20123 download OK and sleep 5 sec
output json file data_invoice_20124.json OK
20124 download OK and sleep 5 sec
output json file data_invoice_20125.json OK
20125 download OK and sleep 5 sec
output json file data_invoice_20130.json OK
20130 download OK an

# 參考資料
# https://morvanzhou.github.io/tutorials/data-manipulation/scraping/3-01-requests/

# http://zwindr.blogspot.com/2016/08/python-requests.html